![arangodb](https://github.com/arangodb/interactive_tutorials/blob/master/notebooks/img/ArangoDB_logo.png?raw=1)

# AQL CRUD Tutorial

<a href="https://colab.research.google.com/github/arangodb/interactive_tutorials/blob/master/notebooks/AqlCrudTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an introduction to ArangoDB’s query language AQL, built around a small dataset of characters from the novel and fantasy drama television series Game of Thrones (as of season 1). It includes character traits in two languages, some family relations, and last but not least a small set of filming locations, which makes for an interesting mix of data to work with.

In addition to the python interface, you can interact with ArangoDB using its web interface to manage collections and execute the queries.

# Setup 

Before getting started with ArangoDB we need to prepare our environment and create a temporary database on ArangoDB's managed Service Oasis.

In [ ]:
%%capture
!git clone -b oasis_connector --single-branch https://github.com/arangodb/interactive_tutorials.git
!rsync -av interactive_tutorials/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [ ]:
import json
import requests
import sys
import oasis
import time

from pyArango.connection import *
from arango import ArangoClient

Create the temporary database:

In [3]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName="AQLCrudTutorial", credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')

# Connect to the temp database
conn = oasis.connect(login)
db = conn[login["dbName"]] 

Requesting new temp credentials.
Temp database ready to use.


In [4]:
print("https://{}:{}".format(login["hostname"], login["port"]))
print("Username: " + login["username"])
print("Password: " + login["password"])
print("Database: " + login["dbName"])

https://tutorials.arangodb.cloud:8529
Username: TUT677r5rs67wqorrr3mhgye
Password: TUTjemn0tt7g48prxzwveweh8
Database: TUT15bh8ysi60h9bn5a2ysvpf


Feel free to use to above URL to checkout the UI!

##  Creating Collections

Before we can insert documents with AQL, we need a place to put them in – a collection. Collections can be managed via the web interface, arangosh or a driver. It is not possible to do so with AQL however.

In [5]:
db.createCollection(name="Characters")

ArangoDB collection name: Characters, id: 14020870, type: document, status: loaded

##  Creating and Reading Documents

The syntax for creating a new document is `INSERT document INTO collectionName`. The document is an object like you may know it from JavaScript or JSON, which is comprised of attribute key and value pairs. The quotes around the attribute keys are optional in AQL. Keys are always character sequences (strings), whereas attribute values can have different types:

* null
* boolean (true, false)
* number (integer and floating point)
* string
* array
* object

Name and surname of the character document we inserted are both string values. The alive state uses a boolean. Age is a numeric value. The traits are an array of strings. The entire document is an object.

In [6]:
insert_query = """
INSERT {
    "name": "Ned",
    "surname": "Stark",
    "alive": true,
    "age": 41,
    "traits": ["A","H","C","N","P"]
} INTO Characters
"""

db.AQLQuery(insert_query)

*Note: You can also execute all the queries from the ArangoDB Web UI which you can access using the link and login information above.*

![ui](https://github.com/joerg84/ArangoDBUniversity/blob/master/img/query_ui.png?raw=1)

Let us check whether the insert was sucessfull querying the `Characters` collections. The syntax of the loop is `FOR variableName IN collectionName`. For each document in the collection, c is assigned a document, which is then returned as per the loop body.

In [7]:
all_characters = """
FOR c IN Characters
    RETURN c
"""

query_result = db.AQLQuery(all_characters, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

{'_key': '18020800', '_id': 'Characters/18020800', '_rev': '_apkT3FC---', 'age': 41, 'alive': True, 'name': 'Ned', 'surname': 'Stark', 'traits': ['A', 'H', 'C', 'N', 'P']}



The document features the four attributes we stored, plus three more added by the database system. Each document needs a unique `_key`, which identifies it within a collection. The `_id` is a computed property, a concatenation of the collection name, a forward slash `/` and the document key. It uniquely identies a document within a database. `_rev` is a revision ID managed by the system. Older revisions of a document cannot be accessed.
Document keys can be provided by the user upon document creation, or a unique value is assigned automatically. It can not be changed later. All three system attributes starting with an underscore `_` are read-only.

Next, let us add some more characters. The `LET` keyword defines a variable with name data and an array of objects as value, so `LET variableName = valueExpression` and the expression being a literal array definition like `[ {...}, {...}, ... ]`.

`FOR variableName IN expression` is used to iterate over each element of the data array. In each loop, one element is assigned to the variable `d`. This variable is then used in the `INSERT` statement instead of a literal object definition. What is does is basically:

In [8]:
insert_query = """
LET data = [
    { "name": "Robert", "surname": "Baratheon", "alive": false, "traits": ["A","H","C"] },
    { "name": "Jaime", "surname": "Lannister", "alive": true, "age": 36, "traits": ["A","F","B"] },
    { "name": "Catelyn", "surname": "Stark", "alive": false, "age": 40, "traits": ["D","H","C"] },
    { "name": "Cersei", "surname": "Lannister", "alive": true, "age": 36, "traits": ["H","E","F"] },
    { "name": "Daenerys", "surname": "Targaryen", "alive": true, "age": 16, "traits": ["D","H","C"] },
    { "name": "Jorah", "surname": "Mormont", "alive": false, "traits": ["A","B","C","F"] },
    { "name": "Petyr", "surname": "Baelish", "alive": false, "traits": ["E","G","F"] },
    { "name": "Viserys", "surname": "Targaryen", "alive": false, "traits": ["O","L","N"] },
    { "name": "Jon", "surname": "Snow", "alive": true, "age": 16, "traits": ["A","B","C","F"] },
    { "name": "Sansa", "surname": "Stark", "alive": true, "age": 13, "traits": ["D","I","J"] },
    { "name": "Arya", "surname": "Stark", "alive": true, "age": 11, "traits": ["C","K","L"] },
    { "name": "Robb", "surname": "Stark", "alive": false, "traits": ["A","B","C","K"] },
    { "name": "Theon", "surname": "Greyjoy", "alive": true, "age": 16, "traits": ["E","R","K"] },
    { "name": "Bran", "surname": "Stark", "alive": true, "age": 10, "traits": ["L","J"] },
    { "name": "Joffrey", "surname": "Baratheon", "alive": false, "age": 19, "traits": ["I","L","O"] },
    { "name": "Sandor", "surname": "Clegane", "alive": true, "traits": ["A","P","K","F"] },
    { "name": "Tyrion", "surname": "Lannister", "alive": true, "age": 32, "traits": ["F","K","M","N"] },
    { "name": "Khal", "surname": "Drogo", "alive": false, "traits": ["A","C","O","P"] },
    { "name": "Tywin", "surname": "Lannister", "alive": false, "traits": ["O","M","H","F"] },
    { "name": "Davos", "surname": "Seaworth", "alive": true, "age": 49, "traits": ["C","K","P","F"] },
    { "name": "Samwell", "surname": "Tarly", "alive": true, "age": 17, "traits": ["C","L","I"] },
    { "name": "Stannis", "surname": "Baratheon", "alive": false, "traits": ["H","O","P","M"] },
    { "name": "Melisandre", "alive": true, "traits": ["G","E","H"] },
    { "name": "Margaery", "surname": "Tyrell", "alive": false, "traits": ["M","D","B"] },
    { "name": "Jeor", "surname": "Mormont", "alive": false, "traits": ["C","H","M","P"] },
    { "name": "Bronn", "alive": true, "traits": ["K","E","C"] },
    { "name": "Varys", "alive": true, "traits": ["M","F","N","E"] },
    { "name": "Shae", "alive": false, "traits": ["M","D","G"] },
    { "name": "Talisa", "surname": "Maegyr", "alive": false, "traits": ["D","C","B"] },
    { "name": "Gendry", "alive": false, "traits": ["K","C","A"] },
    { "name": "Ygritte", "alive": false, "traits": ["A","P","K"] },
    { "name": "Tormund", "surname": "Giantsbane", "alive": true, "traits": ["C","P","A","I"] },
    { "name": "Gilly", "alive": true, "traits": ["L","J"] },
    { "name": "Brienne", "surname": "Tarth", "alive": true, "age": 32, "traits": ["P","C","A","K"] },
    { "name": "Ramsay", "surname": "Bolton", "alive": true, "traits": ["E","O","G","A"] },
    { "name": "Ellaria", "surname": "Sand", "alive": true, "traits": ["P","O","A","E"] },
    { "name": "Daario", "surname": "Naharis", "alive": true, "traits": ["K","P","A"] },
    { "name": "Missandei", "alive": true, "traits": ["D","L","C","M"] },
    { "name": "Tommen", "surname": "Baratheon", "alive": true, "traits": ["I","L","B"] },
    { "name": "Jaqen", "surname": "H'ghar", "alive": true, "traits": ["H","F","K"] },
    { "name": "Roose", "surname": "Bolton", "alive": true, "traits": ["H","E","F","A"] },
    { "name": "The High Sparrow", "alive": true, "traits": ["H","M","F","O"] }
]

FOR d IN data
    INSERT d INTO Characters
"""

db.AQLQuery(insert_query)

As before let us check the `Characters` collection, but this time only return each characters name:

In [9]:
all_characters_names = """
FOR c IN Characters
    RETURN c.name
"""

query_result = db.AQLQuery(all_characters_names, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

Ned

Robert

Jaime

Catelyn

Cersei

Daenerys

Jorah

Petyr

Viserys

Jon

Sansa

Arya

Robb

Theon

Bran

Joffrey

Sandor

Tyrion

Khal

Tywin

Davos

Samwell

Stannis

Melisandre

Margaery

Jeor

Bronn

Varys

Shae

Talisa

Gendry

Ygritte

Tormund

Gilly

Brienne

Ramsay

Ellaria

Daario

Missandei

Tommen

Jaqen

Roose

The High Sparrow



# Updating Documents

**Spoiler Warning!**

According to our Ned Stark document, he is alive. When we get to know that he died, we need to change the alive attribute. Let us modify the existing document. For this we first identify the above `_key` attribute.

In [10]:
find_ned_query = """
FOR c IN Characters
    FILTER c.name == "Ned"
    RETURN c._key
"""

neds_document_key = None
query_result = db.AQLQuery(find_ned_query, rawResults=True)
for doc in  query_result:
    print("_key: " + str(doc))
    neds_document_key = doc
    print()

_key: 18020800



Using `key` we can update an existing document:

In [11]:
kill_ned = """
UPDATE @key 
WITH { alive: false} 
IN Characters
"""
bindVars = {'key': neds_document_key}
db.AQLQuery(kill_ned, rawResults=False, batchSize=1, bindVars=bindVars)


find_ned_query = """
FOR c IN Characters
    FILTER c.name == "Ned"
    RETURN c
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

{'_key': '18020800', '_id': 'Characters/18020800', '_rev': '_apkT4Kq--_', 'age': 41, 'alive': False, 'name': 'Ned', 'surname': 'Stark', 'traits': ['A', 'H', 'C', 'N', 'P']}



We could have also replaced the entire document content, using `REPLACE` instead of `UPDATE`:

In [12]:
kill_ned = """
REPLACE @key WITH {
    name: "Ned",
    surname: "Stark",
    alive: false,
    age: 41,
    traits: ["A","H","C","N","P"]
} IN Characters
"""
bindVars = {'key': neds_document_key}
db.AQLQuery(kill_ned, rawResults=False, batchSize=1, bindVars=bindVars)

# Check the updated Ned document 
find_ned_query = """
FOR c IN Characters
    FILTER c.name == "Ned"
    RETURN c
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

{'_key': '18020800', '_id': 'Characters/18020800', '_rev': '_apkT4lq--_', 'age': 41, 'alive': False, 'name': 'Ned', 'surname': 'Stark', 'traits': ['A', 'H', 'C', 'N', 'P']}



We could again use the `FOR` loop construct from before to update all characters:

In [13]:
season_query = """
FOR c IN Characters
    UPDATE c WITH { season: 1 } IN Characters
"""
db.AQLQuery(season_query, rawResults=False)
    
# Retrieve all characters and season they appeared    
all_characters_names_season = """
FOR c IN Characters
    RETURN {"Name" : c.name, "Season" : c.season}
"""

query_result = db.AQLQuery(all_characters_names_season, rawResults=True)
for doc in  query_result:
    print(doc)
    print()

{'Name': 'Robert', 'Season': 1}

{'Name': 'Jaime', 'Season': 1}

{'Name': 'Catelyn', 'Season': 1}

{'Name': 'Cersei', 'Season': 1}

{'Name': 'Daenerys', 'Season': 1}

{'Name': 'Jorah', 'Season': 1}

{'Name': 'Petyr', 'Season': 1}

{'Name': 'Viserys', 'Season': 1}

{'Name': 'Jon', 'Season': 1}

{'Name': 'Sansa', 'Season': 1}

{'Name': 'Arya', 'Season': 1}

{'Name': 'Robb', 'Season': 1}

{'Name': 'Theon', 'Season': 1}

{'Name': 'Bran', 'Season': 1}

{'Name': 'Joffrey', 'Season': 1}

{'Name': 'Sandor', 'Season': 1}

{'Name': 'Tyrion', 'Season': 1}

{'Name': 'Khal', 'Season': 1}

{'Name': 'Tywin', 'Season': 1}

{'Name': 'Davos', 'Season': 1}

{'Name': 'Samwell', 'Season': 1}

{'Name': 'Stannis', 'Season': 1}

{'Name': 'Melisandre', 'Season': 1}

{'Name': 'Margaery', 'Season': 1}

{'Name': 'Jeor', 'Season': 1}

{'Name': 'Bronn', 'Season': 1}

{'Name': 'Varys', 'Season': 1}

{'Name': 'Shae', 'Season': 1}

{'Name': 'Talisa', 'Season': 1}

{'Name': 'Gendry', 'Season': 1}

{'Name': 'Ygritte', '

Note, that here we customized the `RETURN` to return a json document consisting of name and season.

# Delete Documents

To fully remove documents from a collection, there is the `REMOVE` operation. It works similar to the other modification operations, yet without a WITH clause:

In [14]:
remove_ned = """
REMOVE @key IN Characters
"""
bindVars = {'key': neds_document_key}
try:
    db.AQLQuery(remove_ned, rawResults=False, batchSize=1, bindVars=bindVars)
except:
    print("Ned already removed.")

# Check the updated Ned document, should be empty
find_ned_query = """
FOR c IN Characters
    FILTER c.name == "Ned"
    RETURN c
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)
if len(query_result) == 0 :
    print("Ned not found.")

Ned not found.


As you might have already guessed we can again use a `FOR` loop if we want to perform this operation for the entire collection:

In [15]:
remove_all = """
FOR c IN Characters
    REMOVE c IN Characters
"""
db.AQLQuery(remove_all, rawResults=False)
    
all_characters_names = """
FOR c IN Characters
    RETURN c
"""

query_result = db.AQLQuery(all_characters_names, rawResults=True)
if len(query_result) == 0 :
    print("No characters left.")

No characters left.


# Next Steps

To continue playing and learning with ArangoDB, you can:

* [Get a 2 week free Trial with the ArangoDB Cloud](https://cloud.arangodb.com/home?utm_source=Basic%20AQL%20CRUD&utm_medium=Github&utm_campaign=ArangoDB%20University)
* [Download ArangoDB](https://www.arangodb.com/download-major/)
* Keep Learning at the ArangoDB [Training Center](https://www.arangodb.com/arangodb-training-center/)

*Note: Recall that this notebook uses a temporary instance which will be autodeleted!*

# Further Links

* https://www.arangodb.com/docs/stable/aql/tutorial.html